In [25]:
import pandas as pd
from sqlalchemy import create_engine

In [26]:
employ_file= "DP_LIVE_Employdata.csv"
GDP_file="DP_LIVE_GDPdata.csv"

In [27]:
employ_df = pd.read_csv(employ_file)
employ_df = employ_df.loc[employ_df['MEASURE'] == 'PC_WKGPOP'] #removing the measure = THND_PER; keeping only 0-100 perc. rate
employ_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
163,AUS,EMP,MEN,PC_WKGPOP,A,1979,82.10677,NaN
164,AUS,EMP,MEN,PC_WKGPOP,A,1980,82.25813,NaN
165,AUS,EMP,MEN,PC_WKGPOP,A,1981,82.32400,NaN
166,AUS,EMP,MEN,PC_WKGPOP,A,1982,80.50681,NaN
167,AUS,EMP,MEN,PC_WKGPOP,A,1983,76.92610,NaN


In [28]:
GDP_df=pd.read_csv(GDP_file)
GDP_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,GDP,TOT,MLN_USD,A,1960,25036.1668,NaN
1,AUS,GDP,TOT,MLN_USD,A,1961,25326.3822,NaN
2,AUS,GDP,TOT,MLN_USD,A,1962,27913.2057,NaN
3,AUS,GDP,TOT,MLN_USD,A,1963,30385.7232,NaN
4,AUS,GDP,TOT,MLN_USD,A,1964,32695.8783,NaN


In [29]:
#creating the new dataframe with arequired columns
GDP_new_df=GDP_df[["LOCATION","INDICATOR","SUBJECT","TIME","Value"]]
#GDP_new_df.head()


# Renaming the column names
GDP_new_df=GDP_new_df.rename(columns={"LOCATION":"Country","TIME":"Year","Value":"GDP Value"})
GDP_new_df.head()

,Country,INDICATOR,SUBJECT,Year,GDP Value
0,AUS,GDP,TOT,1960,25036.1668
1,AUS,GDP,TOT,1961,25326.3822
2,AUS,GDP,TOT,1962,27913.2057
3,AUS,GDP,TOT,1963,30385.7232
4,AUS,GDP,TOT,1964,32695.8783


In [40]:
#creating the new dataframe with arequired columns
employ_new_df=employ_df[["LOCATION","INDICATOR","SUBJECT","TIME","Value"]]
employ_new_df.head()


# Renaming the column names
employ_new_df=employ_new_df.rename(columns={"LOCATION":"Country","TIME":"Year","Value":"Percentage of Employement"})
employ_new_df.head()

,Country,INDICATOR,SUBJECT,Year,Percentage of Employement
163,AUS,EMP,MEN,1979,82.10677
164,AUS,EMP,MEN,1980,82.25813
165,AUS,EMP,MEN,1981,82.32400
166,AUS,EMP,MEN,1982,80.50681
167,AUS,EMP,MEN,1983,76.92610


In [41]:
# Dropping the values where subject != Total (Men; Women)
#GDP_new_df = GDP_new_df.loc[GDP_new_df['SUBJECT'] == "TOT"]
employ_new_df = employ_new_df.loc[employ_new_df['SUBJECT'] == "TOT"]
employ_new_df.head()

,Country,INDICATOR,SUBJECT,Year,Percentage of Employement
203,AUS,EMP,TOT,1979,64.38335
204,AUS,EMP,TOT,1980,65.17445
205,AUS,EMP,TOT,1981,65.36742
206,AUS,EMP,TOT,1982,64.16441
207,AUS,EMP,TOT,1983,61.94022


In [42]:
employ_new_df=employ_new_df[["Country","Year","Percentage of Employement"]]
employ_new_df.head()

,Country,Year,Percentage of Employement
203,AUS,1979,64.38335
204,AUS,1980,65.17445
205,AUS,1981,65.36742
206,AUS,1982,64.16441
207,AUS,1983,61.94022


In [43]:
#Employement has 2016-Q2 & 2016-12 data - need to get grouped, averaged out and replaced on yearly basis

employ_new_df['Year'] = pd.to_datetime(employ_new_df['Year']).apply(lambda x : pd.Period(x, freq='M').year)
employ_new_df.head()

,Country,Year,Percentage of Employement
203,AUS,1979,64.38335
204,AUS,1980,65.17445
205,AUS,1981,65.36742
206,AUS,1982,64.16441
207,AUS,1983,61.94022


In [44]:
GDP_new_df=GDP_new_df[["Country","Year","GDP Value"]]
GDP_new_df.head()

,Country,Year,GDP Value
0,ARG,2004,247418.443175
1,ARG,2005,277710.485069
2,ARG,2006,308920.358148
3,ARG,2007,345624.182414
4,ARG,2008,366612.679462


In [45]:
employ_new_df=employ_new_df.loc[employ_new_df['Year'] >= 2000]
GDP_new_df=GDP_new_df.loc[GDP_new_df['Year']>= 2000]
GDP_new_df.head()
employ_new_df.head()

,Country,Year,Percentage of Employement
224,AUS,2000,69.09614
225,AUS,2001,69.00098
226,AUS,2002,69.35914
227,AUS,2003,70.01125
228,AUS,2004,70.29926


In [46]:
employ_new_df= employ_new_df.groupby(['Country','Year'], as_index=False).agg({'Percentage of Employement':'mean'})
GDP_new_df= GDP_new_df.groupby(['Country','Year'], as_index=False).agg({'GDP Value':'mean'})
GDP_new_df.head()
employ_new_df.head()

,Country,Year,Percentage of Employement
0,AUS,2000,69.085338
1,AUS,2001,69.009075
2,AUS,2002,69.332712
3,AUS,2003,69.985809
4,AUS,2004,70.286239


In [47]:
combined_df= pd.merge(GDP_new_df, employ_new_df, on = ['Year', 'Country'], how='inner')
combined_df.head()

,Country,Year,GDP Value,Percentage of Employement
0,AUS,2000,283010.859598,69.085338
1,AUS,2001,298581.305211,69.009075
2,AUS,2002,315021.091451,69.332712
3,AUS,2003,333644.526686,69.985809
4,AUS,2004,354669.272247,70.286239


In [49]:
combined_df=combined_df.round({'GDP Value': 0, 'Percentage of Employement': 0})
combined_df.head()

,Country,Year,GDP Value,Percentage of Employement
0,AUS,2000,283011.0,69.0
1,AUS,2001,298581.0,69.0
2,AUS,2002,315021.0,69.0
3,AUS,2003,333645.0,70.0
4,AUS,2004,354669.0,70.0


In [59]:
combined_df = combined_df.rename(columns={"Country":"country","Year":"year", "GDP Value": "gdp_value", "Percentage of Employement" :"percentage_of_employement"})
combined_df.head()

,country,year,gdp_value,percentage_of_employement
0,AUS,2000,283011.0,69.0
1,AUS,2001,298581.0,69.0
2,AUS,2002,315021.0,69.0
3,AUS,2003,333645.0,70.0
4,AUS,2004,354669.0,70.0


In [63]:
connection_string = "postgres:sanasahel@localhost:5432/ETL-Employement_db"
engine = create_engine(f'postgresql://{connection_string}')

In [64]:
engine.table_names()


['combined_df']

In [66]:
combined_df.to_sql(name='combined_df', con=engine, if_exists='append', index=False)